In [ ]:
!pip install tf_slim
!pip install gdown
!pip install --upgrade --no-cache-dir gdown
!pip install recommenders
!pip install rankfm


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

from rankfm.rankfm import RankFM
from rankfm.evaluation import hit_rate, reciprocal_rank, discounted_cumulative_gain, precision, recall, diversity

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from datetime import datetime
import os
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from scipy import sparse
from scipy.sparse import csc_matrix
from sklearn.decomposition import TruncatedSVD
import random
random.seed(0)
np.random.seed(0)

In [ ]:

from datetime import datetime, timedelta

from recommenders.datasets.python_splitters import (
    python_chrono_split
   
)

In [ ]:
SEED = 42
np.random.seed(SEED)

## Analyse Dataset

In [ ]:
!gdown https://drive.google.com/uc?id=1ccoCOvxD2hFm1ejOVRDlT2SivkoNtC6V

Downloading...
From: https://drive.google.com/uc?id=1ccoCOvxD2hFm1ejOVRDlT2SivkoNtC6V
To: /content/ml-100k.zip
100% 4.92M/4.92M [00:00<00:00, 330MB/s]


In [ ]:
#!unzip "ml-100k.zip"  -d  "/content"

In [ ]:
!mv ml-100k/* ./

mv: cannot stat 'ml-100k/*': No such file or directory


In [ ]:
dateparse = lambda x: datetime.utcfromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S')

rating_df = pd.read_csv('u.data', sep='\t', 
                        names=['userID', 'itemID', 'rating', 'timestamp'], 
                       )

item_df = pd.read_csv('u.item', sep='|', encoding='latin-1',
                    names=['itemID', 'movie', 'release_date', 'video_release_date', 'imdb_url', 'unknown', 'action', 
                           'adventure', 'animation', 'childrens', 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 
                           'film_noir', 'horror', 'musical', 'mystery', 'romance', 'sci_fi', 'thriller', 'war', 'western'])

user_df = pd.read_csv('u.user', sep='|', encoding='latin-1',
                     names=['userID', 'age', 'gender', 'occupation', 'zip_code'])


In [ ]:
unique_users = rating_df.userID.nunique()
unique_items = rating_df.itemID.nunique()

print("interactions shape: {}".format(rating_df.shape))
print("interactions unique users: {}".format(rating_df.userID.nunique()))
print("interactions unique items: {}".format(rating_df.itemID.nunique()))

print("user features users:", rating_df.userID.nunique())
print("item features items:", rating_df.itemID.nunique())

interactions shape: (100000, 4)
interactions unique users: 943
interactions unique items: 1682
user features users: 943
item features items: 1682


In [ ]:
sparsity = 1 - (len(rating_df) / (unique_users * unique_items))
print("interaction matrix sparsity: {}%".format(round(100 * sparsity, 1)))

interaction matrix sparsity: 93.7%


In [ ]:
np.random.seed(42)
rating_df['random'] = np.random.random(size=len(rating_df))
test_pct = 0.25
train_mask = rating_df['random'] <  (1 - test_pct)
valid_mask = rating_df['random'] >= (1 - test_pct)

interactions_train = rating_df[train_mask][['userID', 'itemID']]
interactions_valid = rating_df[valid_mask][['userID', 'itemID']]

train_users = np.sort(interactions_train.userID.unique())
valid_users = np.sort(interactions_valid.userID.unique())
cold_start_users = set(valid_users) - set(train_users)

train_items = np.sort(interactions_train.itemID.unique())
valid_items = np.sort(interactions_valid.itemID.unique())
cold_start_items = set(valid_items) - set(train_items)

print("train shape: {}".format(interactions_train.shape))
print("valid shape: {}".format(interactions_valid.shape))

print("train users: {}".format(len(train_users)))
print("valid users: {}".format(len(valid_users)))
print("cold-start users: {}".format(cold_start_users))

print("train items: {}".format(len(train_items)))
print("valid items: {}".format(len(valid_items)))
print("cold-start items: {}".format(cold_start_items))

train shape: (75055, 2)
valid shape: (24945, 2)
train users: 943
valid users: 942
cold-start users: set()
train items: 1651
valid items: 1453
cold-start items: {1665, 1541, 1417, 1546, 1675, 1559, 1572, 677, 1575, 1325, 1582, 957, 1348, 1604, 1606, 1486, 1363, 852, 1624, 857, 1373, 1636, 1641, 1647, 1520, 1649, 1398, 1654, 1657, 1533, 1406}


In [ ]:
user_df = user_df[user_df.userID.isin(train_users)]
item_df = item_df[item_df.itemID.isin(train_items)]
user_df.shape, item_df.shape

((943, 5), (1651, 24))

Train model

In [ ]:
model = RankFM(factors=20, loss='warp', max_samples=30, alpha=0.01, sigma=0.1, learning_rate=0.10, learning_schedule='invscaling')
model

In [ ]:
%%time
model.fit(interactions_train, epochs=20, verbose=True)


training epoch: 0
log likelihood: -42857.7890625

training epoch: 1
log likelihood: -43229.0390625

training epoch: 2
log likelihood: -41505.609375

training epoch: 3
log likelihood: -41019.80859375

training epoch: 4
log likelihood: -40367.19140625

training epoch: 5
log likelihood: -39891.4296875

training epoch: 6
log likelihood: -39533.078125

training epoch: 7
log likelihood: -39157.46875

training epoch: 8
log likelihood: -38812.23828125

training epoch: 9
log likelihood: -38544.03125

training epoch: 10
log likelihood: -38358.01953125

training epoch: 11
log likelihood: -38190.5703125

training epoch: 12
log likelihood: -38004.46875

training epoch: 13
log likelihood: -37995.96875

training epoch: 14
log likelihood: -38034.44140625

training epoch: 15
log likelihood: -37878.83984375

training epoch: 16
log likelihood: -37803.23828125

training epoch: 17
log likelihood: -37784.28125

training epoch: 18
log likelihood: -37641.69140625

training epoch: 19
log likelihood: -37757.19

In [ ]:
valid_recommendations = model.recommend(valid_users, n_items=10, filter_previous=True, cold_start='nan')
valid_recommendations.head()

,0,1,2,3,4,5,6,7,8,9
1,181,566,70,191,204,22,98,250,68,174
2,286,258,269,275,328,100,15,690,333,50
3,300,313,258,302,690,286,748,289,895,301
4,288,258,328,313,748,286,269,307,332,690
5,432,121,217,67,174,449,501,1,559,183


In [ ]:
k = 20

In [ ]:
most_popular = interactions_train.groupby('itemID')['userID'].count().sort_values(ascending=False)[:k]
most_popular

itemID
50     432
100    385
258    384
181    376
288    370
294    369
286    360
1      338
300    325
121    323
127    322
174    319
56     307
98     301
7      301
117    294
313    277
237    276
172    271
222    269
Name: userID, dtype: int64

In [ ]:
test_user_items = interactions_valid.groupby('userID')['itemID'].apply(set).to_dict()
test_user_items = {key: val for key, val in test_user_items.items() if key in set(train_users)}

base_hrt = np.mean([int(len(set(most_popular.index) & set(val)) > 0)                       for key, val in test_user_items.items()])
base_pre = np.mean([len(set(most_popular.index) & set(val)) / len(set(most_popular.index)) for key, val in test_user_items.items()])
base_rec = np.mean([len(set(most_popular.index) & set(val)) / len(set(val))                for key, val in test_user_items.items()])

print("number of test users: {}".format(len(test_user_items)))
print("baseline hit rate: {:.3f}".format(base_hrt))
print("baseline precision: {:.3f}".format(base_pre))
print("baseline recall: {:.3f}".format(base_rec))

number of test users: 942
baseline hit rate: 0.844
baseline precision: 0.112
baseline recall: 0.126


### Evaluation

In [ ]:
%%time
model_hit_rate = hit_rate(model, interactions_valid, k=k)
model_reciprocal_rank = reciprocal_rank(model, interactions_valid, k=k)
model_dcg = discounted_cumulative_gain(model, interactions_valid, k=k)
model_precision = precision(model, interactions_valid, k=k)
model_recall = recall(model, interactions_valid, k=k)

CPU times: user 2.21 s, sys: 17.1 ms, total: 2.23 s
Wall time: 2.2 s


In [ ]:
print("hit_rate: {:.3f}".format(model_hit_rate))
print("reciprocal_rank: {:.3f}".format(model_reciprocal_rank))
print("dcg: {:.3f}".format(model_dcg, 3))
print("precision: {:.3f}".format(model_precision))
print("recall: {:.3f}".format(model_recall))

hit_rate: 0.908
reciprocal_rank: 0.284
dcg: 0.861
precision: 0.122
recall: 0.177


### Similar items

In [ ]:
random_user = np.random.choice(interactions_valid.userID.unique())
print("random user: {}".format(random_user))
random_user_recs = valid_recommendations.loc[random_user]
random_user_recs = item_df[item_df.itemID.isin(random_user_recs)].set_index('itemID').loc[random_user_recs]
random_user_recs

random user: 868


,movie,release_date,video_release_date,imdb_url,unknown,action,adventure,animation,childrens,comedy,...,fantasy,film_noir,horror,musical,mystery,romance,sci_fi,thriller,war,western
itemID,,,,,,,,,,,,,,,,,,,,,
186,"Blues Brothers, The (1980)",01-Jan-1980,NaN,http://us.imdb.com/M/title-exact?Blues%20Broth...,0,1,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
204,Back to the Future (1985),01-Jan-1985,NaN,http://us.imdb.com/M/title-exact?Back%20to%20t...,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
229,Star Trek III: The Search for Spock (1984),01-Jan-1984,NaN,http://us.imdb.com/M/title-exact?Star%20Trek%2...,0,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
172,"Empire Strikes Back, The (1980)",01-Jan-1980,NaN,http://us.imdb.com/M/title-exact?Empire%20Stri...,0,1,1,0,0,0,...,0,0,0,0,0,1,1,0,1,0
175,Brazil (1985),01-Jan-1985,NaN,http://us.imdb.com/M/title-exact?Brazil%20(1985),0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
474,Dr. Strangelove or: How I Learned to Stop Worr...,01-Jan-1963,NaN,http://us.imdb.com/M/title-exact?Dr.%20Strange...,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
559,Interview with the Vampire (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Interview%20w...,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
79,"Fugitive, The (1993)",01-Jan-1993,NaN,"http://us.imdb.com/M/title-exact?Fugitive,%20T...",0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
219,"Nightmare on Elm Street, A (1984)",01-Jan-1984,NaN,http://us.imdb.com/M/title-exact?Nightmare%20o...,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [ ]:
most_similar_items = model.similar_items(919)
most_similar_items = item_df.set_index('itemID').loc[most_similar_items]
most_similar_items

,movie,release_date,video_release_date,imdb_url,unknown,action,adventure,animation,childrens,comedy,...,fantasy,film_noir,horror,musical,mystery,romance,sci_fi,thriller,war,western
itemID,,,,,,,,,,,,,,,,,,,,,
475,Trainspotting (1996),19-Jul-1996,NaN,http://us.imdb.com/Title?Trainspotting+(1996),0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1097,"Hate (Haine, La) (1995)",09-Feb-1996,NaN,"http://us.imdb.com/M/title-exact?Haine,%20La%2...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1129,Chungking Express (1994),16-Feb-1996,NaN,http://us.imdb.com/M/title-exact?Chongqing%20S...,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
129,Bound (1996),04-Oct-1996,NaN,http://us.imdb.com/M/title-exact?Bound%20(1996),0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
952,Blue in the Face (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Blue%20in%20t...,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
854,Bad Taste (1987),01-Jan-1987,NaN,http://us.imdb.com/M/title-exact?Bad%20Taste%2...,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
93,Welcome to the Dollhouse (1995),24-May-1996,NaN,http://us.imdb.com/Title?Welcome+to+the+Dollho...,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
7,Twelve Monkeys (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Twelve%20Monk...,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1143,Hard Eight (1996),28-Feb-1997,NaN,http://us.imdb.com/Title?Hard+Eight+(1996),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
